In [1]:
import requests as r
from bs4 import BeautifulSoup as bs, SoupStrainer as ss
import json

## Grabbing all the links to specific movies on IMDB

In [2]:
movie_links = []

page = r.get("https://www.imdb.com/movies-in-theaters/?ref_=nv_tp_inth_1")

soup = bs(page.content, 'html.parser')

headers = soup.find_all('h4')

for header in headers:
    movie_links.append(header.a['href'])

    
print(movie_links)


['/title/tt5113040/', '/title/tt6565702/', '/title/tt6107548/', '/title/tt6964076/', '/title/tt4353250/', '/title/tt10208198/', '/title/tt7231342/', '/title/tt7947150/', '/title/tt7721800/', '/title/tt10279010/', '/title/tt8228538/', '/title/tt8580240/', '/title/tt3741700/', '/title/tt6139732/', '/title/tt2066051/', '/title/tt7958736/', '/title/tt6146586/', '/title/tt4154796/', '/title/tt5884052/', '/title/tt1489887/', '/title/tt7752126/', '/title/tt1298644/']


## Going through each link and grabbing necessary info about movie

In [3]:
JSONArray = []

for link in movie_links:
    specPage = r.get("http://www.imdb.com" + link)
    soup = bs(specPage.content, 'html.parser')
    date = soup.find('time')
    if date:
        date = date.text[25:]
        runtime = 0
        runtime += int(date[0])*60
        if len(date) == 29:
            runtime += int(date[3:5])
        elif len(date) == 28:
            runtime += int(date[3])
        
        
        url = soup.find('div', {'class':'mediastrip'}).find_all('a')
        if len(url) > 1 :
            url = url[1]['href']
            print("http://www.imdb.com" + url)
            imagePage = r.get("http://www.imdb.com" + url)
            imgSoup = bs(imagePage.content, 'html.parser')
            imgURL = imgSoup.find_all('script')[1].text[158:-20]
            j = json.loads(imgURL)
            url = j["galleries"][link[7:-1]]['allImages'][1]['src']
            runtime = str(runtime)
            name = soup.find('h1').text[0:-7]
            name = str(name)
            description = soup.find('div', {'class': 'summary_text'}).text[21:]
            description = str(description)
            if description[len(description) - 14] != '.' and description[len(description) - 14] != '?':
                description = description[0: -38]
            print(name, description, runtime, url)
            
            JSONArray.append({"name": name, "description": description, "runtime": runtime, "url": url})


http://www.imdb.com/title/tt5113040/mediaviewer/rm2328323584?context=default
The Secret Life of Pets 2  Continuing the story of Max and his pet friends, following their secret lives after their owners leave them for work or school each day.
             86 https://m.media-amazon.com/images/M/MV5BMmYwYTMyMDYtM2U5Ni00YjU5LTg5ODYtNmUwNGMwYTgzZDBjXkEyXkFqcGdeQXVyNjg2NjQwMDQ@._V1_SX1777_CR0,0,1777,952_AL_.jpg
http://www.imdb.com/title/tt6565702/mediaviewer/rm1733721600?context=default
Dark Phoenix  Jean Grey begins to develop incredible powers that corrupt and turn her into a Dark Phoenix. Now the X-Men will have to decide if the life of a team member is worth more than all the people living in the world.
             113 https://m.media-amazon.com/images/M/MV5BOTVmODA5YzMtZmE4My00NzQ1LWFiOTUtMzNhNzZhMzk0YTQ5XkEyXkFqcGdeQXVyNjg2NjQwMDQ@._V1_SX1777_CR0,0,1777,707_AL_.jpg
http://www.imdb.com/title/tt6107548/mediaviewer/rm1052926976?context=default
Late Night  A late-night talk-show host suspe

http://www.imdb.com/title/tt1298644/mediaviewer/rm1499699713?context=default
The Hustle  Anne Hathaway and Rebel Wilson star as female scam artists, one low rent and the other high class, who team up to take down the men who have wronged them.
             93 https://m.media-amazon.com/images/M/MV5BMTYyNTUwODg4M15BMl5BanBnXkFtZTgwOTY2MDE0NzM@._V1_SX1500_CR0,0,1500,999_AL_.jpg


## Formatting to get clean list of JSON

In [4]:
for js in JSONArray:
    js["name"] = js["name"].rstrip()
    js["description"] = js["description"].rstrip()


In [5]:
jArray = json.dumps(JSONArray)

print(jArray)


[{"name": "The Secret Life of Pets 2", "description": "Continuing the story of Max and his pet friends, following their secret lives after their owners leave them for work or school each day.", "runtime": "86", "url": "https://m.media-amazon.com/images/M/MV5BMmYwYTMyMDYtM2U5Ni00YjU5LTg5ODYtNmUwNGMwYTgzZDBjXkEyXkFqcGdeQXVyNjg2NjQwMDQ@._V1_SX1777_CR0,0,1777,952_AL_.jpg"}, {"name": "Dark Phoenix", "description": "Jean Grey begins to develop incredible powers that corrupt and turn her into a Dark Phoenix. Now the X-Men will have to decide if the life of a team member is worth more than all the people living in the world.", "runtime": "113", "url": "https://m.media-amazon.com/images/M/MV5BOTVmODA5YzMtZmE4My00NzQ1LWFiOTUtMzNhNzZhMzk0YTQ5XkEyXkFqcGdeQXVyNjg2NjQwMDQ@._V1_SX1777_CR0,0,1777,707_AL_.jpg"}, {"name": "Late Night", "description": "A late-night talk-show host suspects that she may soon lose her long-running show.", "runtime": "102", "url": "https://m.media-amazon.com/images/M/MV5BYWZ

## We will be copying and pasting the above JSON into our POST method at route "/movies" which takes in a JSON array of objects, iterates through and adds into database.